In [47]:
import pandas as pd
import numpy as np
import gensim
import pprint
import regex as re
from time import time
from collections import defaultdict
import os
import string
import spacy
from spacy.lang.nl.examples import sentences 
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from collections import Counter
import json
import matplotlib.pyplot as plt
import itertools

In [48]:
#synsets
syns_homo = ['homoseksuele', 'gay', 'flikker','homo', 'homofiel', 'homoseksueel']
syns_neger = ['moor', 'neger', 'negerin'] 
syns_zigeuner = ['roma', 'zigeunerin', 'gipsy', 'zigeuner']
all_syns = syns_homo + syns_neger + syns_zigeuner
all_syns

['homoseksuele',
 'gay',
 'flikker',
 'homo',
 'homofiel',
 'homoseksueel',
 'moor',
 'neger',
 'negerin',
 'roma',
 'zigeunerin',
 'gipsy',
 'zigeuner']

In [49]:
%%time
#voeg alle tekst samen en maak een df
text_df= pd.DataFrame()
rootdir = str(os.getcwd()) + '/textfiles_2107'

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if file.startswith("text"):
            part_df = pd.read_csv(rootdir + "/" + file)
            text_df = text_df.append(part_df)
text_df

CPU times: user 8.29 s, sys: 2.51 s, total: 10.8 s
Wall time: 12.3 s


,Unnamed: 0,article_id,text,decade
0,0,http://resolver.kb.nl/resolve?urn=ABCDDD:01093...,['\nDoorleefd en doorleden\nEen geheel andere ...,1940
1,1,http://resolver.kb.nl/resolve?urn=ABCDDD:01084...,['\nDe Canadees met de broodwagen Sprookje met...,1940
2,2,http://resolver.kb.nl/resolve?urn=ABCDDD:01084...,['\nHOOFDSTUK XX\nAls Torn Hawkins er ook maar...,1940
3,3,http://resolver.kb.nl/resolve?urn=ABCDDD:01088...,['\nVeelzijdigheid\nHet textielbedrijf is inde...,1940
4,4,http://resolver.kb.nl/resolve?urn=ABCDDD:01093...,['\nNieuwe werken dan JAN SLUYTERS\nDE Kunstza...,1940
...,...,...,...,...
57,57,http://resolver.kb.nl/resolve?urn=ABCDDD:01084...,['\nSTERK DEBUUT VAN HEINTJE DAVIDS\n\'van onz...,1960
58,58,http://resolver.kb.nl/resolve?urn=ABCDDD:01087...,['\n\nVINOLIA LOTIONS MET EXQUISE GEUREN TRES ...,1960
59,59,http://resolver.kb.nl/resolve?urn=ABCDDD:01087...,"['\n\nI : H& ■ i Zienl., En gezien worden: Uw ...",1960
60,60,http://resolver.kb.nl/resolve?urn=ABCDDD:01087...,['\nMatroos van „Doorman” voor krijgsraad\n(Va...,1960


In [50]:
%%time
#voeg alle tekst samen met de metadata
meta_df= pd.DataFrame()
rootdir = str(os.getcwd()) + '/metadata_2107'

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if file.startswith("article"):
            part_df = pd.read_csv(rootdir + "/" + file)
            meta_df = meta_df.append(part_df)

CPU times: user 3.22 s, sys: 415 ms, total: 3.63 s
Wall time: 3.76 s


In [51]:
%%time
#voeg alles samen in een df
all_df = meta_df.merge(text_df, on='article_id')
all_df = all_df[['article_id','date','ppn','text','search_term','decade','article_type']]
all_df['search_term'].replace('Roma','roma',inplace=True)
all_df = all_df.sort_values(by='date')
all_df['date'] = pd.to_datetime(all_df['date'])
all_df['text'] = all_df['text'].str.replace('Roma','roma')
all_df.drop_duplicates(subset='article_id', keep="first",inplace=True)
all_df.reset_index(inplace=True)

CPU times: user 1.36 s, sys: 139 ms, total: 1.5 s
Wall time: 1.54 s


,index,article_id,date,ppn,text,search_term,decade,article_type
0,22501,http://resolver.kb.nl/resolve?urn=ddd:11057852...,1940-01-03,832675288,['\nKUNST. Twee Kerst-exposities WERKEN VAN LA...,neger,1940,artikel
1,155564,http://resolver.kb.nl/resolve?urn=ABCDDD:01084...,1940-01-03,412869594,['\nGrote slag bij Salla op komst GEEN MENSEN ...,roma,1940,artikel
2,156018,http://resolver.kb.nl/resolve?urn=ddd:11057852...,1940-01-03,832675288,['\nUIT DE GESCHIEDENIS DEE „CANCELLERÍA”.\nPa...,roma,1940,artikel
3,16,http://resolver.kb.nl/resolve?urn=ABCDDD:01084...,1940-01-04,412869594,['\n\nBURGERLIJKE STAND § Geboren: Benjamin W....,moor,1940,familiebericht
4,155500,http://resolver.kb.nl/resolve?urn=ABCDDD:01084...,1940-01-04,412869594,['\nOngeregeldheden in Leningrad en Moskou ER ...,roma,1940,artikel
...,...,...,...,...,...,...,...,...
113590,2350,http://resolver.kb.nl/resolve?urn=ABCDDD:01082...,1995-12-30,412789353,['\n\nBlijft u liever niet alleen? Bij ons sta...,homo,1990,advertentie
113591,147301,http://resolver.kb.nl/resolve?urn=ABCDDD:01087...,1995-12-30,412869594,"[""\nTELEVISIE\n■ NEDERLAND 1 07.00 NOS: Tekst ...",gay,1990,artikel
113592,133690,http://resolver.kb.nl/resolve?urn=ABCDDD:01087...,1995-12-30,412869594,['\nGoloebjeva - Van Lith (WK-vrouwen 1995)\n$...,roma,1990,artikel
113593,132483,http://resolver.kb.nl/resolve?urn=ABCDDD:01087...,1995-12-30,412869594,['\nVOETBAL\nTRIPOLI: vriendschappelijk toerno...,roma,1990,artikel


In [52]:
#maak df zonder advertenties
no_ads = all_df.loc[all_df['article_type'] == 'artikel']
no_ads['article_type'].unique()

,index,article_id,date,ppn,text,search_term,decade,article_type
0,22501,http://resolver.kb.nl/resolve?urn=ddd:11057852...,1940-01-03,832675288,['\nKUNST. Twee Kerst-exposities WERKEN VAN LA...,neger,1940,artikel
1,155564,http://resolver.kb.nl/resolve?urn=ABCDDD:01084...,1940-01-03,412869594,['\nGrote slag bij Salla op komst GEEN MENSEN ...,roma,1940,artikel
2,156018,http://resolver.kb.nl/resolve?urn=ddd:11057852...,1940-01-03,832675288,['\nUIT DE GESCHIEDENIS DEE „CANCELLERÍA”.\nPa...,roma,1940,artikel
4,155500,http://resolver.kb.nl/resolve?urn=ABCDDD:01084...,1940-01-04,412869594,['\nOngeregeldheden in Leningrad en Moskou ER ...,roma,1940,artikel
5,549,http://resolver.kb.nl/resolve?urn=ddd:11057853...,1940-01-05,832675288,['\nMARKTBERICHTEN.\nPAARDEN EN VEE.\nLEEUWARD...,gay,1940,artikel
...,...,...,...,...,...,...,...,...
113586,1102,http://resolver.kb.nl/resolve?urn=ABCDDD:01082...,1995-12-30,412789353,"[""\nEngeland 2\n08.30 The adventures of Skippy...",gay,1990,artikel
113588,63780,http://resolver.kb.nl/resolve?urn=ABCDDD:01087...,1995-12-30,412869594,"[""\nGOEDE SPELEN, SLECHTE SPELEN door Jan Joos...",homo,1990,artikel
113591,147301,http://resolver.kb.nl/resolve?urn=ABCDDD:01087...,1995-12-30,412869594,"[""\nTELEVISIE\n■ NEDERLAND 1 07.00 NOS: Tekst ...",gay,1990,artikel
113592,133690,http://resolver.kb.nl/resolve?urn=ABCDDD:01087...,1995-12-30,412869594,['\nGoloebjeva - Van Lith (WK-vrouwen 1995)\n$...,roma,1990,artikel


## Save to disk

In [53]:
all_df.to_csv('all_df_2607.csv')
no_ads.to_csv('no_ads_2607.csv')


## Total number of samples (full dataset)

In [54]:
#total
print(len(all_df))

#per woord
print(all_df[['article_id','search_term']].groupby('search_term').agg('count'))

#per krant
print(all_df[['article_id','ppn']].groupby('ppn').agg('count'))

#per decennium
print(all_df[['article_id','decade']].groupby('decade').agg('count'))

#groupby alles
all_df[['article_id','decade','ppn','search_term']].groupby(['decade','ppn','search_term']).agg('count')
len(all_df)

113595
              article_id
search_term             
flikker             1308
gay                16623
gipsy               1472
homo               33817
homofiel            1706
homoseksueel        2642
homoseksuele        5627
moor               16631
neger              11127
negerin             3245
roma               15044
zigeuner            2454
zigeunerin          1899
           article_id
ppn                  
400367629       13212
412789353       25569
412869594       31286
832675288       43528
        article_id
decade            
1940          1762
1950          6656
1960         10809
1970         15057
1980         38996
1990         40315


113595

## Total number of samples (articles only)

In [67]:
#total
print(len(no_ads))
#per woord
print(no_ads[['article_id','search_term']].groupby('search_term').agg('count'))

#per krant
print(no_ads[['article_id','ppn']].groupby('ppn').agg('count'))

#per decennium
print(no_ads[['article_id','decade']].groupby('decade').agg('count'))

#groupby alles
no_ads[['article_id','decade','ppn','search_term']].groupby(['decade','ppn','search_term']).agg('count')
len(no_ads)

61860
              article_id
search_term             
flikker              620
gay                 5282
gipsy               1003
homo                8819
homofiel            1423
homoseksueel        2535
homoseksuele        4651
moor                9924
neger               9606
negerin              875
roma               13184
zigeuner            2137
zigeunerin          1801
           article_id
ppn                  
400367629       11129
412789353       16187
412869594       19830
832675288       14714
        article_id
decade            
1940          1371
1950          5633
1960          9085
1970         11679
1980         19205
1990         14887


61860

## Vocab size pre-preprocessing total tokens + unique tokens

In [72]:
%%time
#tokenize docs

word_tok_all_na =word_tokenize(str(list(no_ads['text'])))
word_tok_unique_na =sent_tokenize(str(list(no_ads['text'])))

#all
print("all word tokens:",len(word_tok_all))
print("all unique word tokens:",len(set(word_tok_unique)))


all word tokens: 84406337
all unique word tokens: 4707318
CPU times: user 8min 8s, sys: 20.3 s, total: 8min 29s
Wall time: 9min 6s


## Preprocessing

In [56]:
#tekst opschonen
def cleaning(docs):
    """lowercases docs and removes newlines. Returns list of docs"""
    clean_docs = []
    for doc in docs:
        doc = doc.replace("\\n", " ")
        doc = doc.lower()
        clean_docs.append(doc)
        #print(doc)
    return clean_docs


## Preprocessing steps:
- Lower
- Remove tekens
- Remove stopwords
- Lemmatize/Stem
- Remove rare tokens


In [57]:
#initialize nlp function
nlp = spacy.load("nl_core_news_sm",disable=['ner', 'parser','tagger','textcat'])
raw_texts =  list(all_df['text'])
raw_texts_na = list(no_ads['text'])

In [58]:
#preprocessing pipeline
def preprocess(texts):
    stopword_list = stopwords.words('dutch')
    texts = [text.replace("\\n", "").strip() for text in texts]
    texts = [text.replace("\\", "").strip() for text in texts]
    docs = nlp.pipe(texts)
    cleaned_lemmas = [[t.lemma_ for t in doc if t.lemma_ not in stopword_list and not t.is_digit and not t.is_punct and len(t) > 3] for doc in docs]
    no_dups = list(cleaned_lemmas for cleaned_lemmas,_ in itertools.groupby(cleaned_lemmas))
    return cleaned_lemmas
    

In [59]:
%%time
tokens_list_post = preprocess(raw_texts)

CPU times: user 1h 37min 58s, sys: 15min 51s, total: 1h 53min 49s
Wall time: 1h 55min 29s


In [60]:
%%time
tokens_list_post_na = preprocess(raw_texts_na)

CPU times: user 26min 19s, sys: 8min 49s, total: 35min 8s
Wall time: 1h 36min 8s


## After preprocessing

In [62]:
%%time
flat_tokens = [word for sublist in tokens_list_post for word in sublist]
flat_tokens_na = [word for sublist in tokens_list_post_na for word in sublist]

CPU times: user 2.24 s, sys: 6.49 s, total: 8.73 s
Wall time: 12.8 s


In [15]:
tokens_df = pd.Series(tokens_list_post)
tokens_df = tokens_df.drop_duplicates()
tokens_df.to_json('tokens_post.json')

In [68]:
#Number of tokens removed
print(len(flat_tokens))# - len(word_tok_all))

#for na
print(len(flat_tokens_na))# - len(word_tok_all)

32913724
19545853


In [69]:
#Total number of tokens
len(flat_tokens)

32913724

In [78]:
#Unique tokens
#print(len(set(flat_tokens)))

print(len(word_tok_all_na))
print(len(word_tok_unique_na))
print(len(flat_tokens_na))
print(len(list(set(flat_tokens_na))))

49214145
2894706
19545853
1760102


all_df